In [13]:
import pandas as pd
import numpy as np

In [14]:
# =============================================================================
# DATA LOADING
# =============================================================================

def load_data(file_path):
    """Load the DVF dataset with proper settings for mixed types."""
    print("=" * 60)
    print("LOADING DATA")
    print("=" * 60)
    df = pd.read_csv(file_path, low_memory=False)
    print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
    return df


In [15]:
# =============================================================================
# FILTERING FUNCTIONS
# =============================================================================

def filter_by_transaction_type(df):
    """Keep only regular sales transactions."""
    print("\n" + "="*40)
    print("FILTERING BY TRANSACTION TYPE")
    print("="*40)
    
    print("Transaction types available:")
    print(df.groupby('nature_mutation').size())
    
    df_sales = df[df['nature_mutation'] == 'Vente'].copy()
    print(f"After sales filter: {df_sales.shape[0]} rows")
    
    return df_sales

def filter_apartments_only(df):
    """Keep only apartments."""
    print("\n" + "="*40)
    print("FILTERING FOR APARTMENTS ONLY")
    print("="*40)
    
    print("Property types available:")
    print(df.groupby('type_local').size())
    
    df_apartments = df[df['type_local'] == 'Appartement'].copy()
    print(f"After apartments filter: {df_apartments.shape[0]} rows")
    
    return df_apartments

def select_key_columns(df):
    """Select only the essential columns for modeling based on EDA conclusions."""
    print("\n" + "="*40)
    print("SELECTING KEY COLUMNS")
    print("="*40)
    
    # Based on our EDA conclusions - 8 features + target
    key_columns = [
        'date_mutation',
        'valeur_fonciere', 
        'code_postal',
        'surface_reelle_bati',
        'nombre_pieces_principales',
        'longitude',
        'latitude',
        'nombre_lots',
        'lot1_surface_carrez',
        'lot2_surface_carrez', 
        'lot3_surface_carrez',
        'lot4_surface_carrez',
        'lot5_surface_carrez'
    ]
    
    df_selected = df[key_columns].copy()
    print(f"Selected {len(key_columns)} key columns")
    
    return df_selected

In [16]:
# =============================================================================
# FEATURE ENGINEERING
# =============================================================================

def remove_missing_values(df):
    """Remove rows with missing values in key columns."""
    print("\n" + "="*40)
    print("REMOVING MISSING VALUES")
    print("="*40)
    
    initial_count = len(df)
    
    # Check missing values
    missing_values = df.isnull().sum()
    if missing_values.sum() > 0:
        print("Missing values found:")
        print(missing_values[missing_values > 0])
        
        df_clean = df.dropna()
        final_count = len(df_clean)
        removed_count = initial_count - final_count
        
        print(f"Rows removed: {removed_count} ({removed_count/initial_count:.1%})")
        print(f"Final dataset: {final_count} rows")
        
        return df_clean
    else:
        print("No missing values found")
        return df

def create_lots_features(df):
    """Create lot-related features based on EDA conclusions."""
    print("\n" + "="*40)
    print("CREATING LOTS FEATURES")
    print("="*40)
    
    # Number of lots with surface data
    lot_columns = ['lot1_surface_carrez', 'lot2_surface_carrez', 'lot3_surface_carrez', 
                   'lot4_surface_carrez', 'lot5_surface_carrez']
    
    df['nb_lots_surface'] = df[lot_columns].notna().sum(axis=1)
    df['a_plusieurs_lots'] = (df['nb_lots_surface'] > 1).astype(int)
    
    # Drop individual lot surface columns (unreliable as per EDA)
    df = df.drop(lot_columns, axis=1)
    
    print("Created features:")
    print("- nb_lots_surface: Number of lots with surface data")
    print("- a_plusieurs_lots: Binary indicator for multiple lots")
    print("- Dropped individual lot surface columns (unreliable)")
    
    return df

def filter_surface_outliers(df):
    """Filter surface outliers based on EDA conclusions: 15-200m²."""
    print("\n" + "="*40)
    print("FILTERING SURFACE OUTLIERS")
    print("="*40)
    
    surface_min = 15  # Based on EDA conclusions
    surface_max = 200
    
    print(f"Surface before filtering:")
    print(f"Min: {df['surface_reelle_bati'].min():.1f}m², Max: {df['surface_reelle_bati'].max():.1f}m²")
    
    initial_count = len(df)
    df_filtered = df[
        (df['surface_reelle_bati'] >= surface_min) & 
        (df['surface_reelle_bati'] <= surface_max)
    ].copy()
    
    removed = initial_count - len(df_filtered)
    print(f"Surface filter ({surface_min}-{surface_max}m²): {initial_count} -> {len(df_filtered)} rows")
    print(f"Removed {removed} outliers ({removed/initial_count:.1%})")
    
    return df_filtered

def filter_pieces_outliers(df):
    """Filter number of pieces outliers based on EDA conclusions: 1-5 pieces."""
    print("\n" + "="*40)
    print("FILTERING PIECES OUTLIERS")
    print("="*40)
    
    pieces_min = 1  # Based on EDA conclusions
    pieces_max = 5
    
    print(f"Pieces before filtering:")
    print(df['nombre_pieces_principales'].value_counts().sort_index())
    
    initial_count = len(df)
    df_filtered = df[
        (df['nombre_pieces_principales'] >= pieces_min) & 
        (df['nombre_pieces_principales'] <= pieces_max)
    ].copy()
    
    removed = initial_count - len(df_filtered)
    print(f"Pieces filter ({pieces_min}-{pieces_max}): {initial_count} -> {len(df_filtered)} rows")
    print(f"Removed {removed} outliers ({removed/initial_count:.1%})")
    
    return df_filtered

def create_target_variable(df):
    """Create the price per square meter target variable."""
    print("\n" + "="*40)
    print("CREATING TARGET VARIABLE (PRIX_M2)")
    print("="*40)
    
    # Calculate price per m2
    df['prix_m2'] = df['valeur_fonciere'] / df['surface_reelle_bati']
    
    # Remove infinite and zero values
    df = df[df['prix_m2'] != np.inf]
    df = df[df['prix_m2'] > 0]
    
    print(f"Prix_m2 statistics:")
    print(df['prix_m2'].describe())
    
    return df

def filter_prix_outliers(df):
    """Filter price outliers based on EDA conclusions: 4000-20000€/m²."""
    print("\n" + "="*40)
    print("FILTERING PRIX_M2 OUTLIERS")
    print("="*40)
    
    prix_min = 4000  # Based on EDA conclusions
    prix_max = 20000
    
    print(f"Prix_m2 before filtering:")
    print(f"Min: {df['prix_m2'].min():.0f}€/m², Max: {df['prix_m2'].max():.0f}€/m²")
    
    initial_count = len(df)
    df_filtered = df[
        (df['prix_m2'] >= prix_min) & 
        (df['prix_m2'] <= prix_max)
    ].copy()
    
    removed = initial_count - len(df_filtered)
    print(f"Prix filter ({prix_min}-{prix_max}€/m²): {initial_count} -> {len(df_filtered)} rows")
    print(f"Removed {removed} outliers ({removed/initial_count:.1%})")
    
    return df_filtered

def extract_time_features(df):
    """Extract time-based features from date_mutation (keeping only year based on EDA)."""
    print("\n" + "="*40)
    print("EXTRACTING TIME FEATURES")
    print("="*40)
    
    # Convert to datetime
    df['date_mutation'] = pd.to_datetime(df['date_mutation'])
    
    # Extract only year (jour, mois had low correlation based on EDA)
    df['annee'] = df['date_mutation'].dt.year
    
    # Drop the date column
    df = df.drop('date_mutation', axis=1)
    
    print("Time features created: annee only (jour, mois had low correlation)")
    print(f"Years in dataset: {sorted(df['annee'].unique())}")
    
    return df

def process_postal_codes(df):
    """Process postal codes to extract arrondissement."""
    print("\n" + "="*40)
    print("PROCESSING POSTAL CODES")
    print("="*40)
    
    print(f"Postal codes before processing:")
    print(f"Min: {df['code_postal'].min()}, Max: {df['code_postal'].max()}")
    print(f"Unique values: {sorted(df['code_postal'].unique())}")
    
    # Filter for valid Paris postal codes only (75001-75020)
    df = df[(df['code_postal'] >= 75001) & (df['code_postal'] <= 75020)].copy()
    print(f"After filtering valid Paris postal codes: {len(df)} rows")
    
    # Extract arrondissement from postal code
    df['arrondissement'] = df['code_postal'].astype(int) - 75000
    
    # Double check arrondissement values are valid (1-20)
    invalid_arrond = df[(df['arrondissement'] < 1) | (df['arrondissement'] > 20)]
    if len(invalid_arrond) > 0:
        print(f"WARNING: Found {len(invalid_arrond)} invalid arrondissements")
        print(invalid_arrond['arrondissement'].unique())
        # Remove invalid arrondissements
        df = df[(df['arrondissement'] >= 1) & (df['arrondissement'] <= 20)].copy()
    
    # Drop old postal code column
    df = df.drop('code_postal', axis=1)
    
    print("Arrondissement feature created")
    print("Arrondissements distribution:")
    print(df['arrondissement'].value_counts().sort_index())
    
    return df
    
    return df

In [17]:
# =============================================================================
# MAIN CLEANING PIPELINE
# =============================================================================

def clean_data_pipeline(df):
    """Complete data cleaning pipeline based on EDA conclusions."""
    print("\n" + "=" * 60)
    print("PARIS REAL ESTATE DATA CLEANING PIPELINE")
    print("BASED ON EDA CONCLUSIONS")
    print("=" * 60)
    
    # 1. Filter for sales only
    df = filter_by_transaction_type(df)
    
    # 2. Keep apartments only
    df = filter_apartments_only(df)
    
    # 3. Select key columns
    df = select_key_columns(df)
    
    # 4. Create lots features (before removing missing values)
    df = create_lots_features(df)
    
    # 5. Remove missing values (after lots processing)
    df = remove_missing_values(df)
    
    # 6. Filter surface outliers (15-200m²)
    df = filter_surface_outliers(df)
    
    # 7. Filter pieces outliers (1-5 pieces)
    df = filter_pieces_outliers(df)
    
    # 8. Create target variable
    df = create_target_variable(df)
    
    # 9. Filter price outliers (4000-20000€/m²)
    df = filter_prix_outliers(df)
    
    # 10. Extract time features (year only)
    df = extract_time_features(df)
    
    # 11. Process postal codes (create arrondissement)
    df = process_postal_codes(df)
    
    print("\n" + "=" * 60)
    print("CLEANING COMPLETE - READY FOR MODELING")
    print("=" * 60)
    print(f"Final dataset: {df.shape[0]} rows, {df.shape[1]} columns")
    print("\nFinal features:")
    for col in df.columns:
        print(f"- {col}")
    
    return df

In [18]:
df = load_data("full_paris_20_24.csv")
df_clean = clean_data_pipeline(df)
# Save model-ready dataset
output_file = "model_paris_20_24.csv"
df_clean.to_csv(output_file, index=False)
print(f"\n✅ Model-ready dataset saved: {output_file}")
print(f"Final shape: {df_clean.shape}")

LOADING DATA
Dataset loaded: 414451 rows, 40 columns

PARIS REAL ESTATE DATA CLEANING PIPELINE
BASED ON EDA CONCLUSIONS

FILTERING BY TRANSACTION TYPE
Transaction types available:
nature_mutation
Adjudication                             886
Echange                                 3089
Expropriation                             19
Vente                                 406010
Vente en l'état futur d'achèvement      4354
Vente terrain à bâtir                     93
dtype: int64
After sales filter: 406010 rows

FILTERING FOR APARTMENTS ONLY
Property types available:
type_local
Appartement                                 196255
Dépendance                                  176663
Local industriel. commercial ou assimilé     28241
Maison                                        2569
dtype: int64
After apartments filter: 196255 rows

SELECTING KEY COLUMNS
Selected 13 key columns

CREATING LOTS FEATURES
Created features:
- nb_lots_surface: Number of lots with surface data
- a_plusieurs_lots: Binary